In [ ]:
import narwhals as nw
import pandas as pd
import polars as pl
import datetime

from narwhals.typing import FrameT

In [ ]:

def dataframe_init_dispatch(
    dataframe_dict: dict,
    library: str,
) -> FrameT:
    """
    Initialize a DataFrame using either Pandas or Polars library based on the specified library name.

    Parameters:
    dataframe_dict (dict): A dictionary where keys are column names and values are lists of column data.
    library (str): The name of the library to use for DataFrame creation. Should be either "pandas" or "polars".

    Returns:
    pl.DataFrame | pd.DataFrame: A DataFrame object from the specified library.

    Raises:
    ValueError: If the `library` parameter is not "pandas" or "polars".
    """

    pandas_df = pd.DataFrame(dataframe_dict)

    # narwhals only supports pyarrow date type, so convert
    for col in pandas_df:
        if pd.api.types.infer_dtype(pandas_df[col]) == "date":
            pandas_df[col] = pandas_df[col].astype("date32[pyarrow]")

        # ensure datetimes setup as expected
        if pd.api.types.is_datetime64_any_dtype(pandas_df[col]):
            pandas_df[col] = pd.to_datetime(pandas_df[col])

    if library == "pandas":
        return pandas_df

    if library == "polars":
        polars_df = pl.DataFrame(dataframe_dict)

        return _align_pandas_and_polars_dtypes(pandas_df, polars_df)

    library_error_message = (
        "The library parameter should be either 'pandas' or 'polars'."
    )
    raise ValueError(library_error_message)

Traceback (most recent call last):
  File "/home/vscode/.vscode-remote/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 5, in <module>
NameError: name 'FrameT' is not defined



In [ ]:
def create_is_between_dates_df_2(library="pandas"):
    """Create df to use in IsBetweenDates tests. Contains 3 columns of 5 datatime values, covers edge cases."""

    df_dict = {
        "a": [
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
        ],
        "b": [
            datetime.datetime(1990, 1, 20, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 2, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 2, 6, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 2, tzinfo=datetime.timezone.utc),
        ],
        "c": [
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
            datetime.datetime(1990, 3, 1, tzinfo=datetime.timezone.utc),
        ],
    }

    return dataframe_init_dispatch(df_dict, library=library)

In [ ]:
df = create_is_between_dates_df_2(library="pandas")
df = nw.from_native(df)

Traceback (most recent call last):
  File "/home/vscode/.vscode-remote/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
  File "<string>", line 31, in create_is_between_dates_df_2
NameError: name 'dataframe_init_dispatch' is not defined



In [ ]:
df.to_native().head()

                          a                         b                         c
0 1990-02-01 00:00:00+00:00 1990-01-20 00:00:00+00:00 1990-03-01 00:00:00+00:00
1 1990-02-01 00:00:00+00:00 1990-02-01 00:00:00+00:00 1990-03-01 00:00:00+00:00
2 1990-02-01 00:00:00+00:00 1990-02-02 00:00:00+00:00 1990-03-01 00:00:00+00:00
3 1990-02-01 00:00:00+00:00 1990-02-06 00:00:00+00:00 1990-03-01 00:00:00+00:00
4 1990-02-01 00:00:00+00:00 1990-03-01 00:00:00+00:00 1990-03-01 00:00:00+00:00


In [ ]:
df.with_row_index()

┌───────────────────────────────────────┐
|          Narwhals DataFrame           |
|---------------------------------------|
|   index  ...                         c|
|0      0  ... 1990-03-01 00:00:00+00:00|
|1      1  ... 1990-03-01 00:00:00+00:00|
|2      2  ... 1990-03-01 00:00:00+00:00|
|3      3  ... 1990-03-01 00:00:00+00:00|
|4      4  ... 1990-03-01 00:00:00+00:00|
|5      5  ... 1990-03-01 00:00:00+00:00|
|                                       |
|[6 rows x 4 columns]                   |
└───────────────────────────────────────┘


In [ ]:
df.with_row_index('i').with_columns(
    c = nw.when(nw.col('i')==0).then(datetime.datetime(1989, 3, 1, tzinfo=datetime.timezone.utc)).otherwise("c")
).drop('i')

┌───────────────────────────────────────┐
|          Narwhals DataFrame           |
| Use `.to_native` to see native output |
└───────────────────────────────────────┘


In [ ]:
df.with_columns(df.with_row_index().allias('i')).with_columns(
            c = nw.when(nw.head(1)).then(datetime.datetime(1989, 3, 1, tzinfo=datetime.timezone.utc)).otherwise("c")
            ).to_native()

Traceback (most recent call last):
  File "/home/vscode/.vscode-remote/extensions/ms-python.python-2025.2.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
AttributeError: 'DataFrame' object has no attribute 'allias'

